In [ ]:
import numpy as np

In [ ]:
filename = r"Y:\Eric\prediction_test\data\interim\trackmate_tests\care_model_eva-v1-dz400-care_rep1_test.xml"

In [ ]:
with open(filename, 'r') as f:
    data = f.read()

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse(filename)
root = tree.getroot()

In [ ]:
for r in root:
    print(r)

In [ ]:
Model = root[1]

In [ ]:
spots = Model[1]
print(spots)
tracks = Model[2]
print(tracks)

In [ ]:
def getSpotCoord(root, spot_id):
    spot = root.findall(f"./Model/AllSpots/SpotsInFrame/*/[@ID='{spot_id}']")
    assert(len(spot) == 1)
    spot_properties = spot[0].attrib
    
    x, y, z = tuple(spot_properties[k] for k in ['POSITION_X', 'POSITION_Y', 'POSITION_Z'])
    return (x, y, z)

In [ ]:
all_spots = root.findall(f"./Model/AllSpots/SpotsInFrame/*")
#print(all_spots)
#for spot in 

In [ ]:
# Works but is too slow:

"""
tracks = []
for track in tqdm(root.findall(f"./Model/AllTracks/*")):
    X = []
    Y = []
    Z = []
    for edge in track.findall('./*'):
        edge_properties = edge.attrib
        spot_source_id = edge_properties['SPOT_SOURCE_ID']
        #print(spot_source_id)
        x_start, y_start, z_start = getSpotCoord(root, spot_source_id)
        X.append(float(x_start))
        Y.append(float(y_start))
        Z.append(float(z_start))
        
    spot_target_id = edge_properties['SPOT_TARGET_ID']
    x_end, y_end, z_end =  getSpotCoord(root, spot_target_id)
    X.append(float(x_end))
    Y.append(float(y_end))
    Z.append(float(z_end))
    tracks.append([X, Y, Z])
"""

In [ ]:
import pandas as pd
from tqdm import tqdm

IDs = []
POSITION_Xs = []
POSITION_Ys = []
POSITION_Zs = []

for spot in tqdm(all_spots):
    spot_properties = spot.attrib
    
    IDs.append(int(spot_properties['ID']))
    POSITION_Xs.append(float(spot_properties['POSITION_X']))
    POSITION_Ys.append(float(spot_properties['POSITION_Y']))
    POSITION_Zs.append(float(spot_properties['POSITION_Z']))


df = pd.DataFrame({'ID': IDs, 'POSITION_X':POSITION_Xs, 'POSITION_Y':POSITION_Ys, 'POSITION_Z':POSITION_Zs})

In [ ]:
df

In [ ]:
spot_start_IDs = []
spot_target_IDs = []
edge_times = []
track_IDs = []

for track in tqdm(root.findall(f"./Model/AllTracks/*")):
    track_id = (track.attrib['TRACK_ID'])
    for edge in track.findall('./*'):
        edge_properties = edge.attrib
        spot_source_id = edge_properties['SPOT_SOURCE_ID']
        time = edge_properties['EDGE_TIME']
        spot_target_id = edge_properties['SPOT_TARGET_ID']
        spot_start_IDs.append(int(spot_source_id))
        spot_target_IDs.append(int(spot_target_id))
        edge_times.append(float(time))
        track_IDs.append(int(track_id))
        
df_edges = pd.DataFrame({'TRACK_ID': track_IDs, 'SPOT_SOURCE_ID': spot_start_IDs, 'SPOT_TARGET_ID':spot_target_IDs,
                         'EDGE_TIME':edge_times})

df_edges

## Now build the tracks

In [ ]:
df_edges = df_edges.merge(df, left_on='SPOT_SOURCE_ID', right_on='ID')
df_edges = df_edges.merge(df, left_on='SPOT_TARGET_ID', right_on='ID')

In [ ]:
df_edges

In [ ]:
df_edges['Distance'] = np.sqrt(
    (df_edges['POSITION_X_x']*0.061 -  df_edges['POSITION_X_y']*0.061 )**2 +
     (df_edges['POSITION_Y_x']*0.061 -  df_edges['POSITION_Y_y']*0.061 )**2 +
     (df_edges['POSITION_Z_x']*0.1 -  df_edges['POSITION_Z_y']*0.1 )**2
)

In [ ]:
%matplotlib inline
df_edges['Distance'].plot.hist(bins=100, alpha=0.5)

In [ ]:
df_edges['Distance'].describe()

In [ ]:
track = df_edges[df_edges.TRACK_ID==0].sort_values(by='EDGE_TIME')
track

In [ ]:
%matplotlib notebook
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca()
ax = fig.gca(projection='3d')
for track_id in df_edges.TRACK_ID.unique()[:10]:
    track = df_edges[df_edges.TRACK_ID==track_id].sort_values(by='EDGE_TIME')
    ax.plot(track.POSITION_X_x, track.POSITION_Y_x, track.POSITION_Z_x, label='parametric curve')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

"""
ax.set_xlim(200, 700)
ax.set_ylim(300, 800)
ax.set_zlim(0, 100)
"""

## Analyze spot statistics

In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv(r"Y:\Eric\prediction_test\data\interim\trackmate_tests\All Spots statistics.csv")

In [ ]:
np.sum(df.TRACK_ID == 'None'), len(df)

## Question: How to relate this with the vtk file?